К текущему моменту мы научились обучать простейшую линейную регрессию: причем научились пользоваться как готовыми методами из sklearn, так и реализовывать поиск решения оптимальных коэффициентов самостоятельно с помощью градиентного спуска.

На последних занятиях мы узнали, что на практике часто приходится прибегать к некоторым хитростям, чтобы оценить, действительно ли модель отлично справляется со своей главной задачей - делать хорошие предсказания на новых данных в будущем, или, как еще говорят, в проде - после запуска.

Для этого было введено понятие так называемой обобщающей способности модели. Оно помогает нам оценить, насколько наша модель подогналась под увиденные данные и насколько она окажется хороша на новых данных.

Обобщающую способность мы научились оценивать с помощью метода Кросс-Валидации.



Построим для старой задачи с предсказанием длительности поездки в такси 2 модели линейной регрессии. Для каждой из них замерим качество на кросс-валидации и на тесте, заранее его отложив.

Считается, что если для K-Fold Кросс-Валидации брать большие значения K (то есть большое количество сплитов для разбиения тренировочной выборки на тренировку-валидацию), то оценка среднеквадратической ошибки будет получаться более справедливой. Так, например, когда K оказывается максимальным, а именно равным количеству объектов в выборке (каждый раз в валидации у нас 1 новый объект), то такой метод Кросс-Валидации еще называют LeaveOneOut.

Главная причина, почему мало где его используют, состоит в дороговизне такого метода. Например, в данной задаче в TAXI_train.csv лежит почти 1,2 млн. объектов. Тогда, чтобы замерить качество на LeaveOneOut валидации, нам пришлось бы обучить 1,2 млн. различных версий одной модели, что, конечно, не вписыватеся в рамки адекватного времени и прочих ресурсов.

Поэтому выберем компромиссное значение в виде 20 фолдов для K-Fold Кросс-Валидации.

Внимание! Для каждой из 2 моделей очевидно было бы справедливым замерять качество на тех же самых объектах, поэтому сплиты лучше зафиксировать сразу.

P.S. Оптимизировать будем MSLE метрику. И сравнивать модели тоже будем по ней будем. Как из домашнего задания про метрики, только без корня! :)

$$\text{MSLE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2$$


### Модель №1 

Для начала посчитаем ошибку на Кросс-Валидации и Тесте для нашей самой базовой модели до вычленения каких-либо признаков, а просто взяв все вещественные колонки.

In [30]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [31]:
#initial_data = pd.read_csv('initial_data.csv', index_col='id')

# Считываем исходные данные из CSV-файла и указываем столбец 'id' как индекс
initial_data = pd.read_csv(r'C:\Users\aefim\github\Start_ML\large\initial_data.csv', index_col='id')

# Задаем список столбцов, которые будут использоваться в дальнейшем
initial_cols = ['vendor_id', 'passenger_count', 'pickup_longitude',
                'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
                'trip_duration']

# Оставляем только выбранные столбцы в исходных данных
initial_data = initial_data[initial_cols]

In [32]:
initial_data.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
id,,,,,,,
id2875421,2,1,-73.982155,40.767937,-73.964630,40.765602,455.0
id2377394,1,1,-73.980415,40.738564,-73.999481,40.731152,663.0
id3858529,2,1,-73.979027,40.763939,-74.005333,40.710087,2124.0
id3504673,2,1,-74.010040,40.719971,-74.012268,40.706718,429.0
id2181028,2,1,-73.973053,40.793209,-73.972923,40.782520,435.0


### ТрюК!

In [33]:
### Замерять будем MSLE. Можно показать, что для оптимизации MSLE,
### Достаточно логарифмировать таргетную переменную, 
### а потом оптимизировать привычные MSE
import numpy as np

# Создаем новый столбец, содержащий логарифм продолжительности поездки
initial_data = initial_data.assign(log_trip_duration=np.log1p(initial_data['trip_duration']))

# Удаляем столбец 'trip_duration'
initial_data = initial_data.drop('trip_duration', axis=1)

### Объяснение:

Пусть имеем модель $a(x)$, обученную на MSE от $y$:
$$
\text{MSE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(y_i - a(x_i)\big)^2 \rightarrow min
$$

Также представим модель $a^*(x)$, обученную на MSE от $\log{(y + 1)}$:
$$
\text{MSE}(X, \log{(y+1)}, a^*) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - a^*(x_i)\big)^2 \rightarrow min
$$

Так как вторая модель старается аппроксимироваться ответы в виде $a^*(x) \approx \log{(y+1)}$, то для того, чтобы получить изначальные таргеты, необходимо выходы $a^*(x)$ проэкспоненцировать в виде следующего отношения: $a(x) = e^{a^*(x)} - 1$, то есть $a^*(x) = \log{(a(x) + 1)}$, где $a^*(x)$ дает логарифмические выходы, а $a(x)$ - изначальные. Подставим данное соотношение в MSE выше, получим:

$$
\text{MSE}(X, \log{(y+1)}, \log{(a(x) + 1)}) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2 \rightarrow min
$$

А это в точности MSLE от $y$:

$$
\text{MSLE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2 \rightarrow min
$$

In [38]:
### Выделим test

from sklearn.model_selection import train_test_split

# Выделяем признаки (X) и целевую переменную (y) из исходных данных
X = initial_data.drop('log_trip_duration', axis=1)
y = initial_data['log_trip_duration']

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

init_X_train = X_train
init_X_test = X_test
init_y_train = y_train
init_y_test = y_test

In [21]:
### Применим K-Fold на оставшуюяся валидационную часть X_train, y_train

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

# Создаем объект, который будет разбивать данные на 20 фолдов для кросс-валидации
splitter = KFold(n_splits=20, shuffle=True, random_state=33)


In [35]:
### Замерьте качество на кросс-валидации!
# Создаем списки для хранения значений ошибки на тестовых и обучающих данных
losses_test = []
losses_train = []

# Для каждого фолда
for train_index, test_index in splitter.split(X):
    # Выделяем обучающие и тестовые данные
    X_train, X_test = X.values[train_index], X.values[test_index]
    Y_train, Y_test = y.values[train_index], y.values[test_index]
    
    # Создаем и обучаем модель линейной регрессии
    model = LinearRegression()
    model.fit(X_train, Y_train)
    
    # Вычисляем среднеквадратичную ошибку на тестовых и обучающих данных и добавляем в списки
    losses_test.append(np.mean((model.predict(X_test)-Y_test)**2))
    losses_train.append(np.mean((model.predict(X_train)-Y_train)**2))

# Находим среднее значение ошибки на тестовых данных
mean_test_loss = np.mean(losses_test)
rounded_test_loss = round(mean_test_loss, 3)
print('Ошибка на тесте: {}'.format(np.mean(rounded_test_loss)))

# Находим среднее значение ошибки на обучающих данных
mean_t_loss = np.mean(losses_train)
rounded_t_loss = round(mean_t_loss, 3)
print('Ошибка на Кросс-Валидации: {}'.format(rounded_t_loss))

Ошибка на тесте: 0.613
Ошибка на Кросс-Валидации: 0.608


Функция cross_validate из библиотеки scikit-learn (sklearn) предоставляет возможность оценки качества модели машинного обучения с помощью перекрестной проверки (cross-validation). Она позволяет оценить метрики качества модели на нескольких независимых тестовых наборах, полученных из исходного набора данных.

cross_validate используется для оценки качества модели машинного обучения с помощью метода кросс-валидации. Этот метод заключается в разделении исходного набора данных на K (обычно выбирается от 3 до 10) равных частей (фолдов), где каждый фолд поочередно используется как тестовый набор, а оставшиеся фолды используются как обучающий набор для обучения модели. После этого оценки качества модели на всех K тестовых наборах усредняются, чтобы получить итоговую оценку качества модели.

Функция cross_validate позволяет задать модель, которую нужно оценить, набор данных, на котором нужно оценить модель, метрику качества модели и количество фолдов. Она возвращает словарь с результатами оценки качества модели, включающий оценки метрик качества на каждом тестовом наборе, среднее значение оценки метрик качества на всех тестовых наборах и стандартное отклонение оценки метрик качества на всех тестовых наборах.

In [45]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

# Создаем объект модели линейной регрессии
model = LinearRegression()

# Производим кросс-валидацию на обучающих данных
cv_results = cross_validate(model, X_train, y_train, scoring='neg_mean_squared_error', cv=splitter)

# Вычисляем среднюю ошибку на тестовых данных
test_error = np.mean(-cv_results['test_score'])

# Выводим значение ошибки на тестовых данных
print('Ошибка на тесте: {}'.format(test_error))

Ошибка на тесте: 0.6134219105328517


In [39]:
### Теперь построим модель на всей тренировочной выборке
### и замерим качество на тесте!

#init_X_train = X_train
#init_X_test = X_test
#init_y_train = y_train
#init_y_test = y_test


# Создаем и обучаем модель линейной регрессии на обучающих данных
model = LinearRegression()
model.fit(init_X_train, init_y_train)

# Вычисляем среднеквадратичную ошибку на тестовых и обучающих данных
MSE_test = np.mean((model.predict(init_X_test)-init_y_test)**2)
MSE_train = np.mean((model.predict(init_X_train)-init_y_train)**2)

# Выводим значения ошибки, округленные до 3 знаков после запятой
print('Ошибка на тесте: {}'.format(round(MSE_test, 3)))
print('Ошибка на всей обучающей выборке: {}'.format(round(MSE_train, 3)))

Ошибка на тесте: 0.606
Ошибка на всей обучающей выборке: 0.609


### Модель №2. Проделаем все то же самое, только для модели с более осознанными признаками, которые удалось получить ранее

In [41]:
# Считываем исходные данные из CSV-файла и указываем столбец 'id' как индекс
processed_data = pd.read_csv(r'C:\Users\aefim\github\Start_ML\large\processed_data.csv', index_col='id')

In [42]:
processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,
id2875421,1,930.399753,0,455.0,1.500479
id2377394,0,930.399753,0,663.0,1.807119
id3858529,1,930.399753,0,2124.0,6.392080
id3504673,1,930.399753,0,429.0,1.487155
id2181028,1,930.399753,0,435.0,1.189925


In [46]:
### Замерять будем MSLE. Можно показать, что для оптимизации MSLE,
### Достаточно логарифмировать таргетную переменную, 
### а потом оптимизировать привычные MSE
import numpy as np

processed_data = processed_data.assign(log_trip_duration=np.log1p(processed_data['trip_duration']))
processed_data = processed_data.drop('trip_duration', axis=1)

In [47]:
X_2 = processed_data.drop('log_trip_duration', axis=1)
y_2 = processed_data['log_trip_duration']

In [48]:
### Важно! Когда сравниваем модели по их качеству
### на валидации и на тесте, не шаффлим данные заново!

test_indexes = X_test.index
train_indexes = X_train.index

X_train_2 = X_2[X_2.index.isin(train_indexes)]
y_train_2 = y_2[y_2.index.isin(train_indexes)]

X_test_2 = X_2[X_2.index.isin(test_indexes)]
y_test_2 = y_2[y_2.index.isin(test_indexes)]

In [52]:
### Замерьте качество на кросс-валидации!
### Your code is here
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

# Создаем объект модели линейной регрессии
model = LinearRegression()

# Производим кросс-валидацию на обучающих данных
cv_results = cross_validate(model, X_train_2, y_train_2, scoring='neg_mean_squared_error', cv=splitter)

# Вычисляем среднюю ошибку на тестовых данных
cross_error = np.mean(-cv_results['test_score'])

# Выводим значение ошибки на тестовых данных
print('Ошибка на кросс-валидации: {}'.format(cross_error))

Ошибка на кросс-валидации: 0.43132577042329395


In [50]:
### Теперь построим модель и замерим качество на тесте!
### Your code is here

# Производим кросс-валидацию на обучающих данных
cv_results = cross_validate(model, X_test_2, y_test_2, scoring='neg_mean_squared_error', cv=splitter)

# Вычисляем среднюю ошибку на тестовых данных
test_error = np.mean(-cv_results['test_score'])

# Выводим значение ошибки на тестовых данных
print('Ошибка на тесте: {}'.format(test_error))

Ошибка на тесте: 0.40643213326024463


In [51]:
### Укажите в ответе на задание 5 среднее качество моделей на валидационных выборках
### и качество модели, обученной на полной тренировочной выборке, на тестовой выборке.
### В качестве разделителя используйте точку, ответ округлите до тысячных.
print('Среднее качество моделей на валидационных выборках: {}'.format(test_error(0.0, 3)))

Среднее качество моделей на валидационных выборках: 0.0


### Какую модель среди двух стоило бы выбрать? Помогла ли нам базовая обработка признаков с первых уроков? 